Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Abstractive Summarization on CNN/DM Dataset using Transformers


### Summary

This notebook demonstrates how to fine tune Transformers for extractive text summarization. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation.




### Before You Start

The running time shown in this notebook is on a Standard_NC24s_v3 Azure Ubuntu Virtual Machine with 4 NVIDIA Tesla V100 GPUs. 
> **Tip**: If you want to run through the notebook quickly, you can set the **`QUICK_RUN`** flag in the cell below to **`True`** to run the notebook on a small subset of the data and a smaller number of epochs. 

Using only 1 NVIDIA Tesla V100 GPUs, 16GB GPU memory configuration,
- for data preprocessing, it takes around 1 minutes to preprocess the data for quick run. Otherwise it takes ~20 minutes to finish the data preprocessing. This time estimation assumes that the chosen transformer model is "distilbert-base-uncased" and the sentence selection method is "greedy", which is the default. The preprocessing time can be significantly longer if the sentence selection method is "combination", which can achieve better model performance.

- for model fine tuning, it takes around 2 minutes for quick run. Otherwise, it takes around ~3 hours to finish. This estimation assumes the chosen encoder method is "transformer". The model fine tuning time can be shorter if other encoder method is chosen, which may result in worse model performance. 

### Additional Notes

* **ROUGE Evalation**: To run rouge evaluation, please refer to the section of compute_rouge_perl in [summarization_evaluation.ipynb](./summarization_evaluation.ipynb) for setup.

* **Distributed Training**:
Please note that the jupyter notebook only allows to use pytorch [DataParallel](https://pytorch.org/docs/master/nn.html#dataparallel). Faster speed and larger batch size can be achieved with pytorch [DistributedDataParallel](https://pytorch.org/docs/master/notes/ddp.html)(DDP). Script [extractive_summarization_cnndm_distributed_train.py](./extractive_summarization_cnndm_distributed_train.py) shows an example of how to use DDP.



In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
## Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs.
QUICK_RUN = True


### Configuration


In [3]:
import os
import shutil
import sys
from tempfile import TemporaryDirectory
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.dataset.cnndm import CNNDMBertSumProcessedData, CNNDMSummarizationDataset
from utils_nlp.eval import compute_rouge_python, compute_rouge_perl
from utils_nlp.models.transformers.abstractive_summarization_bartt5 import SummarizationProcessor

from utils_nlp.models.transformers.datasets import SummarizationDataset
import nltk
from nltk import tokenize

import pandas as pd
import scrapbook as sb
import pprint


### Configuration: choose the transformer model to be used

Several pretrained models have been made available by [Hugging Face](https://github.com/huggingface/transformers). For extractive summarization, the following pretrained models are supported. 

In [4]:
#pd.DataFrame({"model_name": ExtractiveSummarizer.list_supported_models()})

NameError: name 'ExtractiveSummarizer' is not defined

In [4]:
# Transformer model being used
MODEL_NAME = "bart-large-cnn"

In [ ]:
# notebook parameters
# the cache data path during find tuning
from transformers import BartForConditionalGeneration, BartTokenizer
from tqdm import tqdm
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_class = {
    "bart-large-cnn": BartForConditionalGeneration,
    "t5-large":T5ForConditionalGeneration
}
tokenizer_class = {
    "bart-large-cnn": BartTokenizer,
    "t5-large":  T5Tokenizer
}
CACHE_DIR = TemporaryDirectory().name
tokenizer = tokenizer_class[MODEL_NAME].from_pretrained(MODEL_NAME, cache_dir=CACHE_DIR) # b

processor = SummarizationProcessor(tokenizer)

### Data Preprocessing

The dataset we used for this notebook is CNN/DM dataset which contains the documents and accompanying questions from the news articles of CNN and Daily mail. The highlights in each article are used as summary. The dataset consits of ~289K training examples, ~11K valiation examples and ~11K test examples.  


##### [Option 1] Preprocess  data (Please skil this part if you choose to use preprocessed data)
The code in following cell will download the CNN/DM dataset listed at https://github.com/harvardnlp/sent-summary/.

In [7]:
# the data path used to save the downloaded data file
DATA_PATH = TemporaryDirectory().name
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = 1000
if not QUICK_RUN:
    TOP_N = -1

In [8]:
train_dataset, test_dataset = CNNDMSummarizationDataset(top_n=TOP_N, local_cache_path=DATA_PATH, raw=True)

100%|██████████| 489k/489k [00:06<00:00, 70.8kKB/s] 


In [8]:
test_dataset[0].keys()

dict_keys(['src', 'src_txt', 'tgt', 'tgt_txt'])

In [9]:
len(test_dataset)

1000

In [10]:
test_dataset[0]

{'src': "marseille , france -lrb- cnn -rrb- the french prosecutor leading an investigation into the crash of germanwings flight 9525 insisted wednesday that he was not aware of any video footage from on board the plane . marseille prosecutor brice robin told cnn that `` so far no videos were used in the crash investigation . '' he added , `` a person who has such a video needs to immediately give it to the investigators . '' robin 's comments follow claims by two magazines , german daily bild and french paris match , of a cell phone video showing the harrowing final seconds from on board germanwings flight 9525 as it crashed into the french alps . all 150 on board were killed . paris match and bild reported that the video was recovered from a phone at the wreckage site . the two publications described the supposed video , but did not post it on their websites . the publications said that they watched the video , which was found by a source close to the investigation . `` one can hear c

Preprocess the data.

In [10]:

abs_sum_train = processor.preprocess(train_dataset)



In [11]:
abs_sum_train[0]

{'src': "editor 's note : in our behind the scenes series , cnn correspondents share their experiences in covering news and analyze the stories behind the events . here , soledad o'brien takes users inside a jail where many of the inmates are mentally ill . an inmate housed on the `` forgotten floor , '' where many mentally ill inmates are housed in miami before trial . miami , florida -lrb- cnn -rrb- -- the ninth floor of the miami-dade pretrial detention facility is dubbed the `` forgotten floor . '' here , inmates with the most severe mental illnesses are incarcerated until they 're ready to appear in court . most often , they face drug charges or charges of assaulting an officer -- charges that judge steven leifman says are usually `` avoidable felonies . '' he says the arrests often result from confrontations with police . mentally ill people often wo n't do what they 're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more p

In [12]:
abs_sum_test = processor.preprocess(test_dataset)

Exception in thread Thread-24:
Traceback (most recent call last):
  File "/dadendev/anaconda3/envs/nlp_gpu/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/dadendev/anaconda3/envs/nlp_gpu/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/dadendev/anaconda3/envs/nlp_gpu/lib/python3.6/multiprocessing/pool.py", line 463, in _handle_results
    task = get()
  File "/dadendev/anaconda3/envs/nlp_gpu/lib/python3.6/multiprocessing/connection.py", line 251, in recv
    return _ForkingPickler.loads(buf.getbuffer())
  File "/dadendev/anaconda3/envs/nlp_gpu/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 294, in rebuild_storage_fd
    fd = df.detach()
  File "/dadendev/anaconda3/envs/nlp_gpu/lib/python3.6/multiprocessing/resource_sharer.py", line 58, in detach
    return reduction.recv_handle(conn)
  File "/dadendev/anaconda3/envs/nlp_gpu/lib/python3.6/multiprocessing/reduction.py", line 182

AssertionError: 

Process ForkPoolWorker-69:
Process ForkPoolWorker-65:
Process ForkPoolWorker-70:
Process ForkPoolWorker-64:
Process ForkPoolWorker-67:
Process ForkPoolWorker-68:
Process ForkPoolWorker-63:
Process ForkPoolWorker-61:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/dadendev/anaconda3/envs/nlp_gpu/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/dadendev/anaconda3/envs/nlp_gpu/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/dadendev/anaconda3/envs/nlp_gpu/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call la

  File "/dadendev/anaconda3/envs/nlp_gpu/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/dadendev/anaconda3/envs/nlp_gpu/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/dadendev/anaconda3/envs/nlp_gpu/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/dadendev/anaconda3/envs/nlp_gpu/lib/python3.6/multiprocessing/connection.py", line 398, in _send_bytes
    self._send(buf)
KeyboardInterrupt
  File "/dadendev/anaconda3/envs/nlp_gpu/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/dadendev/anaconda3/envs/nlp_gpu/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
  File "/dadendev/anacon

In [ ]:
"""
# save and load preprocessed data
save_path = os.path.join(DATA_PATH, "processed")
torch.save(ext_sum_train, os.path.join(save_path, "train_full.pt"))
torch.save(ext_sum_test, os.path.join(save_path, "test_full.pt"))

"""
# ext_sum_train = torch.load(os.path.join(save_path, "train_full.pt"))
# ext_sum_test = torch.load(os.path.join(save_path, "test_full.pt"))

In [ ]:
len(ext_sum_train)

In [ ]:
len(ext_sum_test)

#### Inspect Data

In [ ]:
ext_sum_train[0]

In [ ]:
ext_sum_train[0].keys()

##### [Option 2] Reuse Preprocessed  data from [BERTSUM Repo](https://github.com/nlpyang/BertSum)

In [ ]:
# the data path used to downloaded the preprocessed data from BERTSUM Repo.
# if you have downloaded the dataset, change the code to use that path where the dataset is.
PROCESSED_DATA_PATH = TemporaryDirectory().name
os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)
#data_path = "./temp_data5/"
#PROCESSED_DATA_PATH = data_path

In [ ]:
if USE_PREPROCSSED_DATA:
    download_path = CNNDMBertSumProcessedData.download(local_path=PROCESSED_DATA_PATH)
    ext_sum_train, ext_sum_test = ExtSumProcessedData().splits(root=download_path, train_iterable=True)
    

### Model training
To start model training, we need to create a instance of ExtractiveSummarizer.
#### Choose the transformer model.
Currently ExtractiveSummarizer support two models:
- distilbert-base-uncase, 
- bert-base-uncase

Potentionally, roberta-based model and xlnet can be supported but needs to be tested.
#### Choose the encoder algorithm.
There are four options:
- baseline: it used a smaller transformer model to replace the bert model and with transformer summarization layer
- classifier: it uses pretrained BERT and fine-tune BERT with **simple logistic classification** summarization layer
- transformer: it uses pretrained BERT and fine-tune BERT with **transformer** summarization layer
- RNN: it uses pretrained BERT and fine-tune BERT with **LSTM** summarization layer

In [ ]:
BATCH_SIZE = 5 # batch size, unit is the number of samples
MAX_POS_LENGTH = 512
if USE_PREPROCSSED_DATA: #if bertsum published data is used
    BATCH_SIZE = 3000 # batch size, unit is the number of tokens
    MAX_POS_LENGTH = 512
    


# GPU used for training
NUM_GPUS = torch.cuda.device_count()

# Encoder name. Options are: 1. baseline, classifier, transformer, rnn.
ENCODER = "transformer"

# Learning rate
LEARNING_RATE=2e-3

# How often the statistics reports show up in training, unit is step.
REPORT_EVERY=100

# total number of steps for training
MAX_STEPS=1e2
# number of steps for warm up
WARMUP_STEPS=5e2
    
if not QUICK_RUN:
    MAX_STEPS=5e4
    WARMUP_STEPS=5e3
 

In [ ]:
summarizer = ExtractiveSummarizer(processor, MODEL_NAME, ENCODER, MAX_POS_LENGTH, CACHE_DIR)

In [ ]:
#"""

summarizer.fit(
            ext_sum_train,
            num_gpus=NUM_GPUS,
            batch_size=BATCH_SIZE,
            gradient_accumulation_steps=2,
            max_steps=MAX_STEPS,
            learning_rate=LEARNING_RATE,
            warmup_steps=WARMUP_STEPS,
            verbose=True,
            report_every=REPORT_EVERY,
            clip_grad_norm=False,
            use_preprocessed_data=USE_PREPROCSSED_DATA
        )

#"""


In [ ]:
summarizer.save_model(
    os.path.join(
        CACHE_DIR,
        "extsum_modelname_{0}_usepreprocess{1}_steps_{2}.pt".format(
            MODEL_NAME, USE_PREPROCSSED_DATA, MAX_STEPS
        ),
    )
)

In [ ]:
# for loading a previous saved model
"""
import torch
model_path = os.path.join(
        CACHE_DIR,
        "extsum_modelname_{0}_usepreprocess{1}_steps_{2}.pt".format(
            MODEL_NAME, USE_PREPROCSSED_DATA, MAX_STEPS
        ))
summarizer = ExtractiveSummarizer(processor, MODEL_NAME, ENCODER, MAX_POS_LENGTH, CACHE_DIR)
summarizer.model.load_state_dict(torch.load(model_path, map_location="cpu"))
"""

### Model Evaluation

[ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)), or Recall-Oriented Understudy for Gisting Evaluation has been commonly used for evaluating text summarization.

In [ ]:
ext_sum_test[0].keys()

In [ ]:
if "segs" in ext_sum_test[0]: # preprocessed_data
    source = [i['src_txt'] for i in ext_sum_test]
    target = ["\n".join(i['tgt_txt'].split("<q>")) for i in ext_sum_test]
else:
    source = []
    temp_target = []
    for i in ext_sum_test:
        source.append(i["src_txt"]) 
        temp_target.append(" ".join(j) for j in i['tgt']) 
    target = [''.join(i) for i in list(temp_target)]

In [ ]:
%%time
sentence_separator = "\n"
prediction = summarizer.predict(ext_sum_test, num_gpus=NUM_GPUS, batch_size=256, sentence_separator=sentence_separator)

In [ ]:
len(prediction)

In [ ]:
rouge_scores = compute_rouge_python(cand=prediction, ref=target)
pprint.pprint(rouge_scores)

In [ ]:
target[0]

In [ ]:
prediction[0]

In [ ]:
source[0]

In [ ]:
# for testing
sb.glue("rouge_2_f_score", rouge_scores['rouge-2']['f'])

## Prediction on a single input sample

In [ ]:
source = """
But under the new rule, set to be announced in the next 48 hours, Border Patrol agents would immediately return anyone to Mexico — without any detainment and without any due process — who attempts to cross the southwestern border between the legal ports of entry. The person would not be held for any length of time in an American facility.

Although they advised that details could change before the announcement, administration officials said the measure was needed to avert what they fear could be a systemwide outbreak of the coronavirus inside detention facilities along the border. Such an outbreak could spread quickly through the immigrant population and could infect large numbers of Border Patrol agents, leaving the southwestern border defenses weakened, the officials argued.
The Trump administration plans to immediately turn back all asylum seekers and other foreigners attempting to enter the United States from Mexico illegally, saying the nation cannot risk allowing the coronavirus to spread through detention facilities and Border Patrol agents, four administration officials said.
The administration officials said the ports of entry would remain open to American citizens, green-card holders and foreigners with proper documentation. Some foreigners would be blocked, including Europeans currently subject to earlier travel restrictions imposed by the administration. The points of entry will also be open to commercial traffic."""

In [ ]:
test_dataset = SummarizationDataset(
    None,
    source=[source],
    source_preprocessing=[tokenize.sent_tokenize],
    word_tokenize=nltk.word_tokenize,
)
processor = ExtSumProcessor(model_name=MODEL_NAME,  cache_dir=CACHE_DIR)
preprocessed_dataset = processor.preprocess(test_dataset)

In [ ]:
preprocessed_dataset[0].keys()

In [ ]:
prediction = summarizer.predict(preprocessed_dataset, num_gpus=0, batch_size=1, sentence_separator="\n")

In [ ]:
prediction

## Clean up temporary folders

In [ ]:
if os.path.exists(DATA_PATH):
    shutil.rmtree(DATA_PATH, ignore_errors=True)
if os.path.exists(CACHE_DIR):
    shutil.rmtree(CACHE_DIR, ignore_errors=True)
if USE_PREPROCSSED_DATA:
    if os.path.exists(PROCESSED_DATA_PATH):
        shutil.rmtree(PROCESSED_DATA_PATH, ignore_errors=True)